## Functions for Working with Openpyxl
JDL, April 2023 (version 4/25/23)

In [3]:
import pandas as pd
import openpyxl
#import openpyxl.utils.dataframe as pyxldf
from openpyxl.styles import Border, Side, Font
import openpyxl.utils as pyxl_util

#### Create two DataFrames to be written to an Excel workbook
Code below explores how to use openpyxl to output formatted versions of the two DataFrames side-by-side on a worksheet. Functions assume DataFrame has a single Pandas index (not multiindex) and single level of column names

In [4]:
pf = 'Example_Data_And_Output.xlsx'
dfTable1 = pd.read_excel(pf, sheet_name='df1')
dfTable1 = dfTable1.set_index('ID_var')

#dfTable2 table in first four rows
dfTable2 = pd.read_excel(pf, sheet_name='df2')
dfTable2 = dfTable2.set_index('ID_var')

print('\n\n', dfTable1, '\n\n', dfTable2, '\n')



           1001   1002   1003   1004
ID_var                             
801     0.0336  0.056  0.056  0.005
802     0.0000  0.000  0.000  0.000
803     0.0000  0.000  0.000  0.000
804     0.0000  0.000  0.000  0.000
805     0.0100  0.050  0.100  0.200 

         Result1  Result2
ID_var                  
801       0.718     30.0
802         NaN      NaN
803         NaN      NaN
804         NaN      NaN
805       0.240     75.0 



#### General Functions for Working with OpenPyXL

In [86]:
def open_workbook(pf):
    """
    openpyxl Open a workbook at specified path+filename
    """
    return openpyxl.load_workbook(pf)

def clear_worksheet(ws):
    """
    Clear Excel worksheet (ws object in an openpyxl wb object)
    JDL 4/21/23
    """
    ws.delete_rows(1, ws.max_row)
    ws.delete_cols(1, ws.max_column)
    return ws

def clear_cell(cell):
    """
    openyxl clear a cell's value, formatting and cell comment/note
    """
    cell.value = None
    cell.font = openpyxl.styles.Font()
    cell.border = openpyxl.styles.Border()
    cell.fill = openpyxl.styles.PatternFill()
    cell.number_format = 'General'
    cell.alignment = openpyxl.styles.Alignment()
    if cell.comment: cell.comment = None
    return cell

def ws_to_df(ws):
    """
    Convert an openpyxl ws to a DataFrame (range index and columns)
    """
    data = ws.values
    df = pd.DataFrame(data)
    return df

def clear_columns(ws, col1, col2):
    """
    Clear specified columns
    """
    for col in range(col1, col2+1):
        for row in range(1, ws.max_row + 1):
            clear_cell(ws.cell(row=row, column=col))
    return ws

def find_string_in_row(ws, irow, sfind):
    """
    Find cell with specified string in specified row
    JDL 4/25/23
    """
    for c in ws[irow]:
        if c.value == sfind: return c
    return None

def find_string_in_col(ws, icol, sfind):
    """
    Find cell with specified string in specified column
    JDL 4/25/23
    """
    for col in ws.iter_cols(min_col=icol, max_col=icol):
        for c in col: 
            if c.value == sfind: return c
    return None

def write_lst_to_rng(ws, cell_home, lstvals, direction='row'):
    """
    Write list of values to cells in specified row or column on openpyxl ws
    direction: either 'row' or 'col'
    JDL 4/25/23
    """
    for i, val in enumerate(lstvals):
        if direction == 'row':
            ws.cell(row=cell.row, column=cell.column+i, value=val)
        elif direction == 'col':
            ws.cell(row=cell.row+i, column=cell.column, value=val)


#### Functions for mocked up Excel sheet
* Create openpyxl wb object with ws populated with some values and formatting
* Create openpyxl wb object with blank ws

In [93]:
def open_wb_and_populate_demo(pf):
    wb = open_workbook(pf)
    ws = wb['output']
    ws = clear_worksheet(ws)
    ws['A1'] = 1.
    ws['B2']  = 'xxx'
    ws['C4'] = 'yyy'
    ws['D6'] = 3.
    ws['D6'].number_format = '0.00'
    return wb, ws

def open_wb_blank_ws(pf):
    wb = open_workbook(pf)
    ws = wb['output']
    ws = clear_worksheet(ws)
    return wb, ws

#### Use ws.dimensions to get range of populated cells on a sheet

In [100]:
#Populate a ws object with some values and formatting
wb, ws = open_wb_and_populate_demo(pf)
print('Populated ws dimensions:', ws.dimensions)
print('\n', ws_to_df(ws), '\n')

print('last row, column:', ws.max_row, ws.max_column, '(Note that openpyxl numbers from 1)\n')

#Blank ws object
wb, ws = open_wb_blank_ws(pf)
print('blank ws dimensions:', ws.dimensions)
print('\n', ws_to_df(ws))

Populated ws dimensions: A1:D6

      0     1     2    3
0  1.0  None  None  NaN
1  NaN   xxx  None  NaN
2  NaN  None  None  NaN
3  NaN  None   yyy  NaN
4  NaN  None  None  NaN
5  NaN  None  None  3.0 

last row, column: 6 4 (Note that openpyxl numbers from 1)

blank ws dimensions: A1:A1

 Empty DataFrame
Columns: []
Index: []


#### Find a specified string in a row or column

In [92]:
wb, ws = open_wb_and_populate_demo(pf)
c = find_string_in_row(ws, 4, 'yyy')
print('yyy was found by find_string_in_row in row 4 in column', c.column)
c = find_string_in_col(ws, 3, 'yyy')
print('yyy was found by find_string_in_col in column 3 row', c.row)

yyy was found by find_string_in_row in row 4 in column 3
yyy was found by find_string_in_col in column 3 row 4


#### Clear entire sheet's contents or specific columns

In [8]:
wb, ws = open_wb_and_populate_demo(pf)
ws = clear_worksheet(ws)
print(ws.dimensions, '\n')
print(ws_to_df(ws))

A1:A1 

Empty DataFrame
Columns: []
Index: []


In [9]:
wb, ws = open_wb_and_populate_demo(pf)
ws = clear_columns(ws, 2,3)
print(ws.dimensions, '\n')
print(ws_to_df(ws))

A1:D6 

     0     1     2    3
0  1.0  None  None  NaN
1  NaN  None  None  NaN
2  NaN  None  None  NaN
3  NaN  None  None  NaN
4  NaN  None  None  NaN
5  NaN  None  None  3.0


In [67]:
ws[ws.dimensions]

((<Cell 'output'.A1>,
  <Cell 'output'.B1>,
  <Cell 'output'.C1>,
  <Cell 'output'.D1>),
 (<Cell 'output'.A2>,
  <Cell 'output'.B2>,
  <Cell 'output'.C2>,
  <Cell 'output'.D2>),
 (<Cell 'output'.A3>,
  <Cell 'output'.B3>,
  <Cell 'output'.C3>,
  <Cell 'output'.D3>),
 (<Cell 'output'.A4>,
  <Cell 'output'.B4>,
  <Cell 'output'.C4>,
  <Cell 'output'.D4>),
 (<Cell 'output'.A5>,
  <Cell 'output'.B5>,
  <Cell 'output'.C5>,
  <Cell 'output'.D5>),
 (<Cell 'output'.A6>,
  <Cell 'output'.B6>,
  <Cell 'output'.C6>,
  <Cell 'output'.D6>))

#### Accessing specific cells

In [11]:
wb, ws = open_wb_and_populate_demo(pf)
print(ws.dimensions)

# Set an openpyxl "used range" and print openpyxl column tuples
used_range = ws[ws.dimensions]
for col in used_range:
    print(col)
    
#Print the last row's row and column number
used_range[-1][0].row, used_range[-1][0].column

A1:D6
(<Cell 'output'.A1>, <Cell 'output'.B1>, <Cell 'output'.C1>, <Cell 'output'.D1>)
(<Cell 'output'.A2>, <Cell 'output'.B2>, <Cell 'output'.C2>, <Cell 'output'.D2>)
(<Cell 'output'.A3>, <Cell 'output'.B3>, <Cell 'output'.C3>, <Cell 'output'.D3>)
(<Cell 'output'.A4>, <Cell 'output'.B4>, <Cell 'output'.C4>, <Cell 'output'.D4>)
(<Cell 'output'.A5>, <Cell 'output'.B5>, <Cell 'output'.C5>, <Cell 'output'.D5>)
(<Cell 'output'.A6>, <Cell 'output'.B6>, <Cell 'output'.C6>, <Cell 'output'.D6>)


(6, 1)

In [12]:
#first and last cell
c = ws.cell(ws.min_row, ws.min_column)
print(c.row, c.column)
c = ws.cell(ws.max_row, ws.max_column)
print(c.row, c.column)

#Change a single value
ws.cell(1,1).value = 'abc'
ws_to_df(ws)

1 1
6 4


,0,1,2,3
0,abc,None,None,NaN
1,None,xxx,None,NaN
2,None,None,None,NaN
3,None,None,yyy,NaN
4,None,None,None,NaN
5,None,None,None,3.0


#### Switching between letter and numeric indices for columns

In [13]:
print(pyxl_util.column_index_from_string('D'))
print(pyxl_util.get_column_letter(4))

4
D


#### Excel Range Iterators
Excel range is defined by openpyxl cell_home and cell_end objects (upper left and lower right corners of range)

In [14]:
def rng_iterator(ws, cell_home, cell_end):
    """
    Return row-wise iterator to iterate over cells in range
    specified by openpyxl home and end cells. Usage: for c in cell_iterator(xxx):
    JDL 4/21/23
    """
    row_start, col_start = cell_home.row, cell_home.column
    row_end, col_end = cell_end.row, cell_end.column

    for row in range(row_start, row_end+1):
        for col in range(col_start, col_end+1):
            cell = ws.cell(row=row, column=col)
            yield cell
            
def rng_iterator_enum(ws, cell_home, cell_end):
    """
    Return row-wise iterator with row, column enumeration to iterate 
    over cells in a range specified by openpyxl home and end cells.
    Usage: for i, j, c in cell_iterator(xxx): where i and j are the
    row and column indices of cells c returned by the generator
    JDL 4/21/23
    """
    start_row, start_col = cell_home.row, cell_home.column
    end_row, end_col,  = cell_end.row, cell_end.column, 
    for i, row in enumerate(range(start_row, end_row+1), start=1):
        for j, col in enumerate(range(start_col, end_col+1), start=1):
            cell = ws.cell(row=row, column=col)
            yield (i, j, cell)

#### rng_iterator demo

In [15]:
wb, ws = open_wb_blank_ws(pf)
ws = clear_worksheet(ws)

#Populate a small, A1:B2 range in the ws object
ws.cell(1,1).value = 'xxx'
ws.cell(2,2).value = 'yyy'
ws.cell(1,2).number_format = '0.00'

#Use the iterator to perform an action (print value and format) on every cell in the range
cell_home, cell_end = ws.cell(row=ws.min_row, column=ws.min_column), ws.cell(row=ws.max_row, column=ws.max_column)
for cell in rng_iterator(ws, cell_home, cell_end):
    print(cell.row, cell.column, cell.value, cell.number_format)

print('\n', ws_to_df(ws))

1 1 xxx General
1 2 None 0.00
2 1 None General
2 2 yyy General

       0     1
0   xxx  None
1  None   yyy


#### rng_iterator_enum demo - Set openpyxl ws.cell objects equal to DataFrame values

In [16]:
wb, ws = open_wb_blank_ws(pf)
cell_home, cell_end = ws.cell(1,1), ws.cell(1+dfTable2.index.size-1, 1+ dfTable2.columns.size-1)

#Use generator to write DataFrame values to cells
for i, j, c in rng_iterator_enum(ws, ws.cell(row=1,column=1), cell_end):
    c.value = dfTable2.values[i-1][j-1]
    
ws_to_df(ws)

,0,1
0,0.718,30.0
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,0.240,75.0


#### Example of making values iterable as list of lists
Not used here but an alternate, useful technique for making df values iterable

In [17]:
print(dfTable2.values.tolist(), '\n')
for lst_row in dfTable2.values.tolist():
    for colvalue in lst_row:
        print(colvalue)

[[0.718, 30.0], [nan, nan], [nan, nan], [nan, nan], [0.24, 75.0]] 

0.718
30.0
nan
nan
nan
nan
nan
nan
0.24
75.0


#### Write a DataFrame to a ws object

In [18]:
def write_dataframe(ws, df, cell_home):
    """
    Write a DataFrame to a specific openpyxl cell on an Excel ws
    cell_home argument is ws.cell for top left data cell in Excel
    JDL 4/23/23
    """
    #Create dict of cell locations for df elements
    d_cells = set_df_openpyxl_cell_locns(ws, df, cell_home)
    
    #Write data, index and column values
    for fn in [write_df_data, write_df_index, write_df_columns]:
        ws = fn(ws, df, d_cells)
    return ws 

def set_df_openpyxl_cell_locns(ws, df, cell_home):
    """
    Set ws.cells for ranges of data, index and columns
    cell_home argument is ws.cell for top left data cell in Excel
    JDL 4/23/23
    """
    row, col = row_col(cell_home)
    d_cells = {'cell_home_data':cell_home}
    d_cells['cell_end_data'] = ws.cell(row + df.index.size - 1, col + df.columns.size - 1)
    d_cells['cell_home_idx'] = ws.cell(row, col - 1)
    d_cells['cell_end_idx'] = ws.cell(row + df.index.size - 1, col - 1)
    d_cells['cell_home_cols'] = ws.cell(row - 1, col)
    d_cells['cell_end_cols'] = ws.cell(row - 1, col + df.columns.size - 1)    
    return d_cells

def row_col(c):
    """
    return openpyxl ws.cell row and column tuple
    JDL 4/23/23
    """
    return c.row, c.column 
    
def write_df_data(ws, df, d_cells):
    """
    Write DataFrame's data values
    JDL 4/23/23
    """
    for i, j, c in rng_iterator_enum(ws, d_cells['cell_home_data'], d_cells['cell_end_data']):
        c.value = df.values[i-1][j-1]        
    return ws
    
def write_df_index(ws, df, d_cells):
    """
    Write DataFrame's index to column adjacent to first data column
    JDL 4/23/23
    """
    #Write index values
    for i, j, c in rng_iterator_enum(ws, d_cells['cell_home_idx'], d_cells['cell_end_idx']):
        c.value = list(df.index)[i-1]
    
    #Write index name as heading above index values
    ws.cell(d_cells['cell_home_idx'].row - 1, d_cells['cell_home_idx'].column).value = df.index.name
    return ws

def write_df_columns(ws, df, d_cells):
    """
    Write DataFrame's column values to row above to first data row
    JDL 4/23/23
    """    
    for i, j, c in rng_iterator_enum(ws, d_cells['cell_home_cols'], d_cells['cell_end_cols']):
        c.value = list(df.columns)[j-1]  
    return ws

#### Demo - Write two dfs to a sheet
* dfTable1 is in columns 0 to 3 (aka Excel columns A to E)
* Column 5 (aka Excel column F( is blank)
* dfTable2 is written to columns 6 to 8 (aka Excel columns G to I)

In [19]:
wb, ws = open_wb_and_populate_demo(pf)
ws = clear_worksheet(ws)
print(ws.dimensions)

ws = write_dataframe(ws, dfTable1, ws.cell(2,2))
ws = write_dataframe(ws, dfTable2, ws.cell(2,8))
ws_to_df(ws)

A1:A1


,0,1,2,3,4,5,6,7,8
0,ID_var,1001.0000,1002.000,1003.000,1004.000,None,ID_var,Result1,Result2
1,801,0.0336,0.056,0.056,0.005,None,801,0.718,30.0
2,802,0.0000,0.000,0.000,0.000,None,802,NaN,NaN
3,803,0.0000,0.000,0.000,0.000,None,803,NaN,NaN
4,804,0.0000,0.000,0.000,0.000,None,804,NaN,NaN
5,805,0.0100,0.050,0.100,0.200,None,805,0.24,75.0


#### Cell borders
* Openpyxl requires creation of a 'Border" object
* Hint: style='xxx' gives error message listing available Side style strings)</br></br>
<left><img src="images/output_border_test.png" alt="" width="300"></left>

In [20]:
#Example write a border to B3 on Demo worksheet

thick_border = Border(left=Side(style='thick'), 
                     right=Side(style='thick'), 
                     top=Side(style='thick'), 
                     bottom=Side(style='thick'))

wb, ws = open_wb_and_populate_demo(pf)
print(ws_to_df(ws))
ws.cell(row=3, column=2).border = thick_border
wb.save('border_test.xlsx')

     0     1     2    3
0  1.0  None  None  NaN
1  NaN   xxx  None  NaN
2  NaN  None  None  NaN
3  NaN  None   yyy  NaN
4  NaN  None  None  NaN
5  NaN  None  None  3.0


#### Functions for applying outline borders to cells and DataFrame outputs to ws objects

In [21]:
def set_openpyxl_border_obj(style_border):
    """
    Create a border style based on style_border='thick', 'thin' etc.
    Use "from openpyxl.styles import Border, Side" to import needed openpyxl attributes
    JDL 4/21/23
    """
    return Border(left=Side(style=style_border),
                  right=Side(style=style_border), 
                  top=Side(style=style_border),
                  bottom=Side(style=style_border))

def set_range_border(ws, cell_home, cell_end, style_border):
    """
    Set borders for an Excel range defined by ws cell_home and cell_end
    JDL 4/21/23
    """
    #Create a Border object for style_border
    border_obj = set_openpyxl_border_obj(style_border)
    
    #Apply the border_obj to each cell in the range
    for c in rng_iterator(ws, cell_home, cell_end):
        c.border = border_obj
        
def set_df_borders(ws, df, cell_home):
    """
    Set borders for an Excel range containing a DataFrame
    JDL 4/21/23
    """
    d_cells = set_df_openpyxl_cell_locns(ws, df, cell_home)
    ws = set_df_data_borders(ws, d_cells, 'thin')
    ws = set_df_index_borders(ws, d_cells, 'thin')
    ws = set_df_cols_borders(ws, d_cells, 'thick')
    return ws

def set_df_data_borders(ws, d_cells, style_border):
    """
    Put border around cells for df data values
    """
    set_range_border(ws, d_cells['cell_home_data'], d_cells['cell_end_data'], style_border)
    return ws

def set_df_index_borders(ws, d_cells, style_border):
    set_range_border(ws, d_cells['cell_home_idx'], d_cells['cell_end_idx'], style_border)
    row = d_cells['cell_home_idx'].row - 1
    col = d_cells['cell_home_idx'].column
    set_range_border(ws, ws.cell(row, col), ws.cell(row, col), style_border)
    return ws

def set_df_cols_borders(ws, d_cells, style_border):
    set_range_border(ws, d_cells['cell_home_cols'], d_cells['cell_end_cols'], style_border)
    return ws

#### Demo - Write two df's to a ws object with border formatting
<left><img src="images/output_border_test2.png" alt="" width="600"></left>

In [22]:
wb, ws = open_wb_blank_ws(pf)
ws = write_dataframe(ws, dfTable1, ws.cell(2,2))
ws = write_dataframe(ws, dfTable2, ws.cell(2,8))
ws = set_df_borders(ws, dfTable1, ws.cell(2,2))
ws = set_df_borders(ws, dfTable2, ws.cell(2,8))
wb.save('border_test2.xlsx')

#### Functions for applying built-in styles to a DataFrame

In [23]:
def set_range_builtin_style(ws, cell_home, cell_end, style_builtin):
    """
    Apply the builtin style to each cell in the range
    JDL 4/25/23
    """
    for c in rng_iterator(ws, cell_home, cell_end):
        c.style = style_builtin
        
def set_df_builtin_styles(ws, df, cell_home, style_data=None, style_idx=None, style_cols=None):
    """
    Set built-in styles for Excel range with a DataFrame
    JDL 4/25/23
    """
    d_cells = set_df_openpyxl_cell_locns(ws, df, cell_home)
    if not style_data is None: ws = set_df_data_builtin_styles(ws, d_cells, style_data)
    if not style_idx is None: ws = set_df_index_builtin_styles(ws, d_cells, style_idx)
    if not style_cols is None: ws = set_df_cols_builtin_styles(ws, d_cells, style_cols)
    return ws

def set_df_data_builtin_styles(ws, d_cells, style_data):
    """
    Set built-in Excel style for df data values
    """
    set_range_builtin_style(ws, d_cells['cell_home_data'], d_cells['cell_end_data'], style_data)
    return ws

def set_df_index_builtin_styles(ws, d_cells, style_idx):
    """
    Set built-in Excel style for df index values
    """    
    set_range_builtin_style(ws, d_cells['cell_home_idx'], d_cells['cell_end_idx'], style_idx)
    return ws

def set_df_cols_builtin_styles(ws, d_cells, style_cols):
    """
    Set built-in Excel style for df column values and index name cell
    """    
    set_range_builtin_style(ws, d_cells['cell_home_cols'], d_cells['cell_end_cols'], style_cols)
    
    #Set index name cell same style as data columns
    row = d_cells['cell_home_idx'].row - 1
    col = d_cells['cell_home_idx'].column
    set_range_builtin_style(ws, ws.cell(row, col), ws.cell(row, col), style_cols)
    return ws

In [39]:
#Comprehensive write DataFrame with borders and applied styles
wb, ws = open_wb_blank_ws(pf)
ws = write_dataframe(ws, dfTable1, ws.cell(2,2))
ws = write_dataframe(ws, dfTable2, ws.cell(2,8))
ws = set_df_borders(ws, dfTable1, ws.cell(2,2))
ws = set_df_borders(ws, dfTable2, ws.cell(2,8))
ws = set_df_builtin_styles(ws, dfTable1, ws.cell(2,2), None, '40 % - Accent1', 'Accent1')
ws = set_df_builtin_styles(ws, dfTable2, ws.cell(2,8), None, '40 % - Accent1', 'Accent1')
#wb.save('border_test2.xlsx')

In [25]:
#Check that *.py library works
if False:
    import util_openpyxl as up
    
    wb, ws = open_wb_blank_ws(pf)
    ws = up.write_dataframe(ws, dfTable1, ws.cell(2,2))
    ws = up.write_dataframe(ws, dfTable2, ws.cell(2,8))
    ws = up.set_df_borders(ws, dfTable1, ws.cell(2,2))
    ws = up.set_df_borders(ws, dfTable2, ws.cell(2,8))
    ws = up.set_df_builtin_styles(ws, dfTable1, ws.cell(2,2), None, '40 % - Accent1', 'Accent1')
    ws = up.set_df_builtin_styles(ws, dfTable2, ws.cell(2,8), None, '40 % - Accent1', 'Accent1')
    #wb.save('border_test2.xlsx')

### Set column widths
* Not resolved how to do this. Roundoff error causes the 0.056 values to have excessive decimal digits --resulting in exaggerated widths
* Possibly could use f-string to format numbers: {number:.3f} before taking len

In [40]:
#### Set DataFrame Columns widths        
def set_range_column_widths(ws, col_start, col_end, width):
    """
    Set a contiguous range of columns (e.g. df.columns) to a specified width
    JDL 4/25/23
    """
    for col in range(col_start, col_end+1):
        letter = pyxl_util.get_column_letter(col)
        ws.column_dimensions[letter].width = width + 0.6
    return ws

In [41]:
col_start, col_end = 2, col_start + dfTable1.columns.size -1 
ws = set_range_column_width(ws, col_start, col_end, 15.)

In [42]:
wb.save('border_test2.xlsx')

#### Autofit columns

In [67]:
for col in ws.columns:
    length = 0
    letter = (pyxl_util.get_column_letter(col[0].column))
    
    for c in col:
        print(c.column, c.row, len(str(c.value)), c.value)
        length = max(length, len(str(c.value)))
            
    print('length', col[0], length)
    if length > 0: ws.column_dimensions[letter].width = length*1.23

1 1 6 ID_var
1 2 3 801
1 3 3 802
1 4 3 803
1 5 3 804
1 6 3 805
length <Cell 'output'.A1> 6
2 1 4 1001
2 2 6 0.0336
2 3 3 0.0
2 4 3 0.0
2 5 3 0.0
2 6 4 0.01
length <Cell 'output'.B1> 6
3 1 4 1002
3 2 20 0.055999999999999994
3 3 3 0.0
3 4 3 0.0
3 5 3 0.0
3 6 4 0.05
length <Cell 'output'.C1> 20
4 1 4 1003
4 2 20 0.055999999999999994
4 3 3 0.0
4 4 3 0.0
4 5 3 0.0
4 6 3 0.1
length <Cell 'output'.D1> 20
5 1 4 1004
5 2 5 0.005
5 3 3 0.0
5 4 3 0.0
5 5 3 0.0
5 6 3 0.2
length <Cell 'output'.E1> 5
6 1 4 None
6 2 4 None
6 3 4 None
6 4 4 None
6 5 4 None
6 6 4 None
length <Cell 'output'.F1> 4
7 1 6 ID_var
7 2 3 801
7 3 3 802
7 4 3 803
7 5 3 804
7 6 3 805
length <Cell 'output'.G1> 6
8 1 7 Result1
8 2 5 0.718
8 3 3 nan
8 4 3 nan
8 5 3 nan
8 6 4 0.24
length <Cell 'output'.H1> 7
9 1 7 Result2
9 2 4 30.0
9 3 3 nan
9 4 3 nan
9 5 3 nan
9 6 4 75.0
length <Cell 'output'.I1> 7


In [78]:
dfTable1.applymap(str)

,1001,1002,1003,1004
ID_var,,,,
801,0.0336,0.055999999999999994,0.055999999999999994,0.005
802,0.0,0.0,0.0,0.0
803,0.0,0.0,0.0,0.0
804,0.0,0.0,0.0,0.0
805,0.01,0.05,0.1,0.2


In [79]:
dfTable1.round(4).applymap(str)

,1001,1002,1003,1004
ID_var,,,,
801,0.0336,0.056,0.056,0.005
802,0.0,0.0,0.0,0.0
803,0.0,0.0,0.0,0.0
804,0.0,0.0,0.0,0.0
805,0.01,0.05,0.1,0.2


In [80]:
dfTable1

,1001,1002,1003,1004
ID_var,,,,
801,0.0336,0.056,0.056,0.005
802,0.0000,0.000,0.000,0.000
803,0.0000,0.000,0.000,0.000
804,0.0000,0.000,0.000,0.000
805,0.0100,0.050,0.100,0.200
